In [18]:
import pandas as pd
import sys, os, lucene, threading, time 
from java.nio.file import Paths
from org.apache.lucene import analysis, document, index, queryparser, search, store
from org.apache.lucene.store import SimpleFSDirectory 
from org.apache.lucene.index import FieldInfo, IndexWriter, IndexWriterConfig ,DirectoryReader,IndexReader,Term
from org.apache.lucene.search import \
    BooleanClause, BooleanQuery, Explanation, PhraseQuery, TermQuery, ScoreMode
from org.apache.lucene.util import Version
from org.apache.lucene.search.similarities import \
ClassicSimilarity,BooleanSimilarity, LMDirichletSimilarity,BM25Similarity,TFIDFSimilarity,LMJelinekMercerSimilarity
from org.apache.lucene.queryparser.classic import QueryParser,MultiFieldQueryParser
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.pylucene.queryparser.classic import \
    PythonQueryParser, PythonMultiFieldQueryParser
from org.apache.lucene.document import LatLonPoint,FloatPoint,IntPoint

### Read exeperiment queries

In [11]:
tmp = pd.read_excel('./experiment_query.xlsx',header=None)

In [16]:
total_queries = list(tmp.iloc[:,1].values)
del tmp

### Lucene Initialization

In [19]:
lucene.initVM()
PATH = './data1/index'
directory =  SimpleFSDirectory(Paths.get(PATH))
ireader = DirectoryReader.open(directory)

model_1 = search.IndexSearcher(ireader)
model_2 = search.IndexSearcher(ireader)

### Set Model

In [21]:
bm25 = BM25Similarity()
tf_idf = ClassicSimilarity()

model_1.setSimilarity(bm25)
model_2.setSimilarity(tf_idf)

In [22]:
multiFiled_query=  PythonMultiFieldQueryParser(['city','attributes','categories','review'],StandardAnalyzer())
multiFiled_query.setDefaultOperator(QueryParser.Operator.AND)


In [28]:
def generate_expResult(model, queries, top_num):
    
    result = [[],[]]
    for each_query in queries:
        
        mrq = multiFiled_query.parse(each_query,['city','attributes','categories','review'],
        [BooleanClause.Occur.SHOULD, BooleanClause.Occur.SHOULD,BooleanClause.Occur.SHOULD,BooleanClause.Occur.SHOULD],
        StandardAnalyzer())
        
        mrq_hits=model.search(mrq,top_num).scoreDocs
        
        for hit in mrq_hits:
            hitDoc = model.doc(hit.doc)
            result[0].append(each_query)
            result[1].append(hitDoc['business_id'])
    return result

In [31]:
res_model1 = generate_expResult(model_1, total_queries, 10)
res_model2 = generate_expResult(model_2, total_queries, 10)

In [33]:
df_model_1 = pd.DataFrame(
    {
        'query':res_model1[0],
        'business_id':res_model1[1]
    }
)

df_model_2 = pd.DataFrame(
    {
        'query':res_model2[0],
        'business_id':res_model2[1]
    }
)